In [1]:
##### import sys
import numpy as np
import random

from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.layouts import gridplot
from bokeh.models import HoverTool
from bokeh.io import output_notebook

# parameters
N = 15  # N x N filterbank for each layer
dims = [100,100] # input_img size
num_filter = N*N+N*N
# delta, sigma2
delta_1=max(dims[0],dims[1])*1.5/(N-1)
sigma2_1=delta_1*delta_1/4 # sigma=delta/2
delta_2=max(dims[0],dims[1])/3/(N-1)
sigma2_2=delta_2*delta_2/4 # sigma=delta/2

# generate blobs
blob_size = [5,4,3,4,2]
height = [0,0,0,0,0]

# top left corner coordinate
cX = [40,20,55,60,90]
cY = [40,25,60,10,80]
k = len(cX)
for t in range(k):
    height[t] = blob_size[t]
width = height # square blobs

# input_img
input_img = np.ones(dims[0]*dims[1])
for i in range(k):
    for j in range(cX[i], cX[i]+width[i]):
        for k in range(cY[i], cY[i]+height[i]):
            input_img[j*dims[1]+k] = 255
        
input_img = np.reshape(input_img,(dims[0],dims[1]))

# attention window
gx = dims[0]/2
gy = dims[1]/2

grid_i = np.arange(N)
mu_x_1 = np.reshape(gx + (grid_i - N / 2 + 0.5) * delta_1, (-1,1))
mu_y_1 = np.reshape(gy + (grid_i - N / 2 + 0.5) * delta_1, (-1,1)) 
mu_x_2 = np.reshape(gx + (grid_i - N / 2 + 0.5) * delta_2, (-1,1))
mu_y_2 = np.reshape(gy + (grid_i - N / 2 + 0.5) * delta_2, (-1,1))
a = np.reshape(np.arange(dims[0]), (1, -1)) 
b = np.reshape(np.arange(dims[1]), (1, -1)) 

mu_x_1_list = np.reshape([mu_x_1]*N,[-1]) 
mu_y_1_list = np.reshape(np.transpose([mu_y_1]*N),[-1]) 
mu_x_2_list = np.reshape([mu_x_2]*N,[-1]) 
mu_y_2_list = np.reshape(np.transpose([mu_y_2]*N),[-1])

Fx_1 = np.exp(-np.square(a - mu_x_1) / (2*sigma2_1)) 
Fy_1 = np.exp(-np.square(b - mu_y_1) / (2*sigma2_1)) 
Fx_2 = np.exp(-np.square(a - mu_x_2) / (2*sigma2_2))
Fy_2 = np.exp(-np.square(b - mu_y_2) / (2*sigma2_2))

# normalize, sum over A and B dims
Fx_1=Fx_1/np.reshape(np.sum(Fx_1,axis=1),(-1,1))
Fy_1=Fy_1/np.reshape(np.sum(Fy_1,axis=1),(-1,1))
Fx_2=Fx_2/np.reshape(np.sum(Fx_2,axis=1),(-1,1))
Fy_2=Fy_2/np.reshape(np.sum(Fy_2,axis=1),(-1,1))

#filter_img
Fxt_1=np.transpose(Fx_1)
np.shape(Fxt_1)
filter_img_1=np.matmul(Fy_1, np.matmul(input_img, Fxt_1))
filter_img_1=np.log(np.reshape(filter_img_1,(N,N)))

Fxt_2=np.transpose(Fx_2)
np.shape(Fxt_2)
filter_img_2=np.matmul(Fy_2, np.matmul(input_img, Fxt_2))
filter_img_2=np.log(np.reshape(filter_img_2,(N,N)))

hover = HoverTool(
        tooltips="""
        <div>
            <div>
                <span style="font-size: 15px; font-weight: bold;">@desc</span>
            </div>
            <div>
                <span style="font-size: 12px;">Location</span>
                <span style="font-size: 12px; color: #320;">($x, $y)</span>
            </div>
        </div>
        """
    )


source = ColumnDataSource(
        data=dict(
            x=[gx],
            y=[gy],
            desc=['Att_window Center'],
        )
)

dot_source_1 = ColumnDataSource(
        data=dict(
            mu_x_1=mu_x_1_list,
            mu_y_1=mu_y_1_list,
            #desc=['Att_window Center'],
        )
)

dot_source_2 = ColumnDataSource(
        data=dict(
            mu_x_2=mu_x_2_list,
            mu_y_2=mu_y_2_list,
            #desc=['Att_window Center'],
        )
)

p = figure(title="Input Image (gx=%-d, gy=%-d)" %(gx,gy), x_range=(-55, 155), y_range=(-55, 155), tools=[hover])
iii = p.image(image=[input_img], x=0, y=0, dw=100, dh=100, palette="Greys256")
p.circle('mu_x_1', 'mu_y_1', size=2, source=dot_source_1, color="orange")
p.circle('mu_x_2', 'mu_y_2', size=2, source=dot_source_2, color="blue")
p.circle('x', 'y', size=5, source=source, color="red")

filter_source = ColumnDataSource(
        data=dict(
            x=[N/2],
            y=[N/2],
            desc=['Filterbank Center'],
        )
)


p_filter_1 = figure(title="Outer Layer Filter Image", x_range=(0, 15), y_range=(0, 15))
iii = p_filter_1.image(image=[filter_img_1], x=0, y=0, dw=15, dh=15, palette="Greys256")
p_filter_1.circle('x', 'y', size=10, source=filter_source, color="red")

p_filter_2 = figure(title="Inner Layer Filter Image", x_range=(0, 15), y_range=(0, 15))
iii = p_filter_2.image(image=[filter_img_2], x=0, y=0, dw=15, dh=15, palette="Greys256")
p_filter_2.circle('x', 'y', size=10, source=filter_source, color="red")

output_notebook()

show(p)
pp = gridplot([[p_filter_1,p_filter_2]], toolbar_location="right")
show(pp)

Loading BokehJS ...

In [2]:
# generate blobs
blob_size = 5
# top left corner coordinate
cX = [25,35,52,63,81]
cY = [35,55,40,36,51]
k = len(cX)
height = blob_size
width = height # square blobs

# input_img
input_img = np.zeros(dims[0]*dims[1])
for i in range(k):
    for j in range(cY[i], cY[i]+height):
        for k in range(cX[i], cX[i]+width):
            input_img[j*dims[1]+k] = 100

i=4

# attention window
gx = 100#cX[i]+2.5#dims[0]/2
gy = 50#cY[i]+2.5#dims[1]/2

for j in range(cY[i], cY[i]+5):
    for k in range(cX[i], cX[i]+5):
        input_img[j*dims[1]+k] = 100
            
input_img = np.reshape(input_img,(dims[0],dims[1]))
grid_i = np.arange(N)
mu_x_1 = np.reshape(gx + (grid_i - N / 2 + 0.5) * delta_1, (-1,1))
mu_y_1 = np.reshape(gy + (grid_i - N / 2 + 0.5) * delta_1, (-1,1)) 
mu_x_2 = np.reshape(gx + (grid_i - N / 2 + 0.5) * delta_2, (-1,1))
mu_y_2 = np.reshape(gy + (grid_i - N / 2 + 0.5) * delta_2, (-1,1))
a = np.reshape(np.arange(dims[0]), (1, -1)) 
b = np.reshape(np.arange(dims[1]), (1, -1)) 

mu_x_1_list = np.reshape([mu_x_1]*N,[-1]) 
mu_y_1_list = np.reshape(np.transpose([mu_y_1]*N),[-1]) 
mu_x_2_list = np.reshape([mu_x_2]*N,[-1]) 
mu_y_2_list = np.reshape(np.transpose([mu_y_2]*N),[-1])


source = ColumnDataSource(
        data=dict(
            x=[gx],
            y=[gy],
            desc=['Att_window Center'],
        )
)

dot_source_1 = ColumnDataSource(
        data=dict(
            mu_x_1=mu_x_1_list,
            mu_y_1=mu_y_1_list,
            #desc=['Att_window Center'],
        )
)

dot_source_2 = ColumnDataSource(
        data=dict(
            mu_x_2=mu_x_2_list,
            mu_y_2=mu_y_2_list,
            #desc=['Att_window Center'],
        )
)

p = figure(title="Input Image (gx=%-d, gy=%-d)" %(gx,gy), x_range=(-55, 155), y_range=(-55, 155))
iii = p.image(image=[input_img], x=0, y=0, dw=100, dh=100, palette="Greys256")
p.circle('mu_x_1', 'mu_y_1', size=2, source=dot_source_1, color="orange")
p.circle('mu_x_2', 'mu_y_2', size=2, source=dot_source_2, color="blue")
p.circle('x', 'y', size=5, source=source, color="red")

filter_source = ColumnDataSource(
        data=dict(
            x=[N/2],
            y=[N/2],
            desc=['Filterbank Center'],
        )
)

output_notebook()

show(p)

Loading BokehJS ...

In [3]:
##### import sys
import numpy as np
import random

from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.layouts import gridplot
from bokeh.models import HoverTool
from bokeh.io import output_notebook

# parameters
N = 15  # N x N filterbank for each layer
dims = [100,100] # input_img size
num_filter = N*N+N*N
# delta, sigma2
delta_1=max(dims[0],dims[1])*1.5/(N-1)
sigma2_1=delta_1*delta_1/4 # sigma=delta/2
delta_2=max(dims[0],dims[1])/3/(N-1)
sigma2_2=delta_2*delta_2/4 # sigma=delta/2

# generate blobs
blob_size = [6,5,4,6,5,6,4,5,6]
height = [0,0,0,0,0,0,0,0,0]
#blob_size = [5,5,5,5,5,5,5,5,5]
#height = [0,0,0,0,0,0,0,0,0]

# top left corner coordinate
cX = [8,17,25,34,44,53,60,70,79]
cY = [35,50,42,59,39,55,49,37,52]
#cX = [8,17,25,33,42,51,60,68,77]
#cY = [47,48,48,47,48,47,47,48,47]


k = len(cX)
for t in range(k):
    height[t] = blob_size[t]
width = height # square blobs

# input_img
input_img = np.ones(dims[0]*dims[1])
for i in range(k):
    for j in range(cY[i], cY[i]+height[i]):
        for k in range(cX[i], cX[i]+width[i]):
            input_img[j*dims[0]+k] = 255

# attention window
gx = 8+3#dims[0]/2
gy = 35+3#dims[1]/2

grid_i = np.arange(N)
mu_x_1 = np.reshape(gx + (grid_i - N / 2 + 0.5) * delta_1, (-1,1))
mu_y_1 = np.reshape(gy + (grid_i - N / 2 + 0.5) * delta_1, (-1,1)) 
mu_x_2 = np.reshape(gx + (grid_i - N / 2 + 0.5) * delta_2, (-1,1))
mu_y_2 = np.reshape(gy + (grid_i - N / 2 + 0.5) * delta_2, (-1,1))
a = np.reshape(np.arange(dims[0]), (1, -1)) 
b = np.reshape(np.arange(dims[1]), (1, -1)) 

mu_x_1_list = np.reshape([mu_x_1]*N,[-1]) 
mu_y_1_list = np.reshape(np.transpose([mu_y_1]*N),[-1]) 
mu_x_2_list = np.reshape([mu_x_2]*N,[-1]) 
mu_y_2_list = np.reshape(np.transpose([mu_y_2]*N),[-1])

Fx_1 = np.exp(-np.square(a - mu_x_1) / (2*sigma2_1)) 
Fy_1 = np.exp(-np.square(b - mu_y_1) / (2*sigma2_1)) 
Fx_2 = np.exp(-np.square(a - mu_x_2) / (2*sigma2_2))
Fy_2 = np.exp(-np.square(b - mu_y_2) / (2*sigma2_2))

# normalize, sum over A and B dims
Fx_1=Fx_1/np.reshape(np.sum(Fx_1,axis=1),(-1,1))
Fy_1=Fy_1/np.reshape(np.sum(Fy_1,axis=1),(-1,1))
Fx_2=Fx_2/np.reshape(np.sum(Fx_2,axis=1),(-1,1))
Fy_2=Fy_2/np.reshape(np.sum(Fy_2,axis=1),(-1,1))

# add_pointer
p_size = 2
gx_idx = np.reshape(int(gx), [1])
gy_idx = np.reshape(int(gy), [1])
i_idx = (gy_idx-int(p_size/2))*dims[0] + (gx_idx-int(p_size/2))
ii_idx = p_size
iii_idx = (gx_idx-int(p_size/2)) + dims[0] - (gx_idx-int(p_size/2)) - p_size
iv_idx = dims[0] - (gx_idx-int(p_size/2)) - int(p_size) + dims[0]*(dims[1] - ((gy_idx-int(p_size/2)) + int(p_size)))
i = np.ones(i_idx)
ii = np.ones(ii_idx)*255 # pointer blob
iii = np.ones(iii_idx)
iv = np.ones(iv_idx)
pointer = np.concatenate([np.concatenate([np.concatenate([np.concatenate([i,ii], 0), iii], 0), ii], 0), iv], 0)
pointer = np.reshape(pointer, [1,-1])
#input_img = input_img*pointer
#for i in range(10000):
    #if input_img[0,i]>255:
        #input_img[0,i]=255
        
input_img = np.reshape(input_img,(dims[0],dims[1]))

#filter_img
Fxt_1=np.transpose(Fx_1)
np.shape(Fxt_1)
filter_img_1=np.matmul(Fy_1, np.matmul(input_img, Fxt_1))
filter_img_1=np.log(np.reshape(filter_img_1,(N,N)))

Fxt_2=np.transpose(Fx_2)
np.shape(Fxt_2)
filter_img_2=np.matmul(Fy_2, np.matmul(input_img, Fxt_2))
filter_img_2=np.log(np.reshape(filter_img_2,(N,N)))

hover = HoverTool(
        tooltips="""
        <div>
            <div>
                <span style="font-size: 15px; font-weight: bold;">@desc</span>
            </div>
            <div>
                <span style="font-size: 12px;">Location</span>
                <span style="font-size: 12px; color: #320;">($x, $y)</span>
            </div>
        </div>
        """
    )


source = ColumnDataSource(
        data=dict(
            x=[gx],
            y=[gy],
            desc=['Att_window Center'],
        )
)

dot_source_1 = ColumnDataSource(
        data=dict(
            mu_x_1=mu_x_1_list,
            mu_y_1=mu_y_1_list,
            #desc=['Att_window Center'],
        )
)

dot_source_2 = ColumnDataSource(
        data=dict(
            mu_x_2=mu_x_2_list,
            mu_y_2=mu_y_2_list,
            #desc=['Att_window Center'],
        )
)

p = figure(title="Input Image (gx=%-d, gy=%-d)" %(gx,gy), x_range=(-80, 200), y_range=(-80, 200), tools=[hover])
iii = p.image(image=[input_img], x=0, y=0, dw=100, dh=100, palette="Greys256")
p.circle('mu_x_1', 'mu_y_1', size=5, source=dot_source_1, color="orange")
p.circle('mu_x_2', 'mu_y_2', size=2, source=dot_source_2, color="blue")
p.circle('x', 'y', size=3, source=source, color="red")



filter_source = ColumnDataSource(
        data=dict(
            x=[N/2],
            y=[N/2],
            desc=['Filterbank Center'],
        )
)


p_filter_1 = figure(title="Outer Layer Filter Image", x_range=(0, 15), y_range=(0, 15))
iii = p_filter_1.image(image=[filter_img_1], x=0, y=0, dw=15, dh=15, palette="Greys256")
p_filter_1.circle('x', 'y', size=10, source=filter_source, color="red")

p_filter_2 = figure(title="Inner Layer Filter Image", x_range=(0, 15), y_range=(0, 15))
iii = p_filter_2.image(image=[filter_img_2], x=0, y=0, dw=15, dh=15, palette="Greys256")
p_filter_2.circle('x', 'y', size=10, source=filter_source, color="red")

output_notebook()

show(p)
pp = gridplot([[p_filter_1,p_filter_2]], toolbar_location="right")
show(pp)



Loading BokehJS ...

In [4]:
gx=0
gy=50
p_size = 2
gx_idx = np.reshape(int(gx), [1])
gy_idx = np.reshape(int(gy), [1])
i_idx = (gy_idx-int(p_size/2))*dims[0] + (gx_idx-int(p_size/2))
ii_idx = p_size
iii_idx = (gx_idx-int(p_size/2)) + dims[0] - (gx_idx-int(p_size/2)) - p_size
iv_idx = dims[0] - (gx_idx-int(p_size/2)) - int(p_size) + dims[0]*(dims[1] - ((gy_idx-int(p_size/2)) + int(p_size)))
i_idx+ii_idx+iii_idx+iv_idx
99 - gx_idx + 100*(99 - gy_idx)

array([4999])

In [5]:
i = np.zeros(i_idx)
ii = np.ones(ii_idx)*255 # pointer blob
iii = np.zeros(iii_idx)
iv = np.zeros(iv_idx)
pointer = np.concatenate([np.concatenate([np.concatenate([np.concatenate([i,ii], 0), iii], 0), ii], 0), iv], 0)
len(pointer)
pointer = np.reshape(pointer, [1,-1])

In [6]:
##### import sys
import numpy as np
import random

from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.layouts import gridplot
from bokeh.models import HoverTool
from bokeh.io import output_notebook

# parameters
N = 15  # N x N filterbank for each layer
dims = [100,100] # input_img size
num_filter = N*N+N*N
# delta, sigma2
delta_1=max(dims[0],dims[1])*1.5/(N-1)
sigma2_1=delta_1*delta_1/4 # sigma=delta/2
delta_2=max(dims[0],dims[1])/3/(N-1)
sigma2_2=delta_2*delta_2/4 # sigma=delta/2

# generate blobs
blob_size = [5,5,5,5,5]
height = [0,0,0,0,0]

# top left corner coordinate
cX = [8,17,25,34,42]
cY = [47,48,48,47,48]
k = len(cX)
for t in range(k):
    height[t] = blob_size[t]
width = height # square blobs

# input_img
input_img = np.ones(dims[0]*dims[1])
for i in range(k):
    for j in range(cY[i], cY[i]+height[i]):
        for k in range(cX[i], cX[i]+width[i]):
            input_img[j*dims[0]+k] = 155

# add_pointer
p_size = 2
gx_idx = np.reshape(int(gx), [1])
gy_idx = np.reshape(int(gy), [1])
i_idx = (gy_idx-int(p_size/2))*dims[0] + (gx_idx-int(p_size/2))
ii_idx = p_size
iii_idx = (gx_idx-int(p_size/2)) + dims[0] - (gx_idx-int(p_size/2)) - p_size
iv_idx = dims[0] - (gx_idx-int(p_size/2)) - int(p_size) + dims[0]*(dims[1] - ((gy_idx-int(p_size/2)) + int(p_size)))
i = np.ones(i_idx)
ii = np.ones(ii_idx)*255 # pointer blob
iii = np.ones(iii_idx)
iv = np.ones(iv_idx)
pointer = np.concatenate([np.concatenate([np.concatenate([np.concatenate([i,ii], 0), iii], 0), ii], 0), iv], 0)
pointer = np.reshape(pointer, [1,-1])
#input_img = input_img*pointer

input_img

array([ 1.,  1.,  1., ...,  1.,  1.,  1.])

In [7]:
##### import sys
import numpy as np
import random

from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.layouts import gridplot
from bokeh.models import HoverTool
from bokeh.io import output_notebook

# parameters
N = 15  # N x N filterbank for each layer
dims = [100,100] # input_img size
num_filter = N*N+N*N
# delta, sigma2
delta_1=max(dims[0],dims[1])*1.5/(N-1)
sigma2_1=delta_1*delta_1/4 # sigma=delta/2
delta_2=max(dims[0],dims[1])/3/(N-1)
sigma2_2=delta_2*delta_2/4 # sigma=delta/2

# generate blobs
blob_size = [5,5,5,5,5]
height = [0,0,0,0,0]

# top left corner coordinate
cX = [8,17,25,34,42]
cY = [47,48,48,47,48]
k = len(cX)
for t in range(k):
    height[t] = blob_size[t]
width = height # square blobs

# input_img
input_img = np.ones(dims[0]*dims[1])
for i in range(k):
    for j in range(cY[i], cY[i]+height[i]):
        for k in range(cX[i], cX[i]+width[i]):
            input_img[j*dims[0]+k] = 155

# attention window
gx = 8#dims[0]/2
gy = 47#dims[1]/2

grid_i = np.arange(N)
mu_x_1 = np.reshape(gx + (grid_i - N / 2 + 0.5) * delta_1, (-1,1))
mu_y_1 = np.reshape(gy + (grid_i - N / 2 + 0.5) * delta_1, (-1,1)) 
mu_x_2 = np.reshape(gx + (grid_i - N / 2 + 0.5) * delta_2, (-1,1))
mu_y_2 = np.reshape(gy + (grid_i - N / 2 + 0.5) * delta_2, (-1,1))
a = np.reshape(np.arange(dims[0]), (1, -1)) 
b = np.reshape(np.arange(dims[1]), (1, -1)) 

mu_x_1_list = np.reshape([mu_x_1]*N,[-1]) 
mu_y_1_list = np.reshape(np.transpose([mu_y_1]*N),[-1]) 
mu_x_2_list = np.reshape([mu_x_2]*N,[-1]) 
mu_y_2_list = np.reshape(np.transpose([mu_y_2]*N),[-1])

Fx_1 = np.exp(-np.square(a - mu_x_1) / (2*sigma2_1)) 
Fy_1 = np.exp(-np.square(b - mu_y_1) / (2*sigma2_1)) 
Fx_2 = np.exp(-np.square(a - mu_x_2) / (2*sigma2_2))
Fy_2 = np.exp(-np.square(b - mu_y_2) / (2*sigma2_2))

# normalize, sum over A and B dims
Fx_1=Fx_1/np.reshape(np.sum(Fx_1,axis=1),(-1,1))
Fy_1=Fy_1/np.reshape(np.sum(Fy_1,axis=1),(-1,1))
Fx_2=Fx_2/np.reshape(np.sum(Fx_2,axis=1),(-1,1))
Fy_2=Fy_2/np.reshape(np.sum(Fy_2,axis=1),(-1,1))

# add_pointer
p_size = 2
gx_idx = np.reshape(int(gx), [1])
gy_idx = np.reshape(int(gy), [1])
i_idx = (gy_idx-int(p_size/2))*dims[0] + (gx_idx-int(p_size/2))
ii_idx = p_size
iii_idx = (gx_idx-int(p_size/2)) + dims[0] - (gx_idx-int(p_size/2)) - p_size
iv_idx = dims[0] - (gx_idx-int(p_size/2)) - int(p_size) + dims[0]*(dims[1] - ((gy_idx-int(p_size/2)) + int(p_size)))
i = np.ones(i_idx)
ii = np.ones(ii_idx)*255 # pointer blob
iii = np.ones(iii_idx)
iv = np.ones(iv_idx)
pointer = np.concatenate([np.concatenate([np.concatenate([np.concatenate([i,ii], 0), iii], 0), ii], 0), iv], 0)
pointer = np.reshape(pointer, [1,-1])
iinput_img = input_img*pointer

np.argmax(iinput_img)
iinput_img[0,4708]

39525.0